# Intra-day Trading with Facebook Prophet

In [8]:
# Libraries
from eod import EodHistoricalData
import pandas as pd
from datetime import datetime, timedelta
import calendar
from tqdm import tqdm
import numpy as np
import random
import plotly.express as px
from prophet import Prophet

# Importing and assigning the api key
with open("../../eodHistoricalData-API.txt", "r") as f:
    api_key = f.read()
    
# EOD Historical Data client
client = EodHistoricalData(api_key)

# Getting Price Data

In [12]:
ago = datetime.utcnow() - timedelta(10)
timestamp_unix = str(calendar.timegm(ago.utctimetuple()))

In [13]:
resp = client.get_prices_intraday('BTC-USD.CC', interval='5m', from_=timestamp_unix)

In [ ]:
def getPrices(ticker, training_days, sent_df, mov_avg):
    """
    Using a sentiment DataFrame to find the first day and trains with the previous
    N days to make predictions that coincide with the sentiment date range.
    
    In order to properly backtest with sentiment, a certain amount of training 
    days need to come before the initial date from the sentiment DF.
    
    Also prepares the price data for FBProphet.
    """
    # The first day from the sentiment DF minus the amount of days to train with and moving avg
    ago = datetime.strptime(
        sent_df.index[0], "%Y-%m-%d"
    ) - timedelta(
        days=training_days+mov_avg
    )
    
    # Getting prices
    prices = pd.DataFrame(
        client.get_prices_eod(
            ticker+"-USD.CC", 
            from_=ago.strftime("%Y-%m-%d")
        )
    )

    # Set index
    prices = prices.set_index('date', drop=True)
    
    # Getting the N Day Moving Average and rounding the values for some light data preprocessing
    prices['MA'] = prices[['open']].rolling(
        window=mov_avg
    ).mean().apply(lambda x: round(x, 2))

    # Dropping Nans
    prices.dropna(inplace=True)

    # Resetting format for FBP
    prices = prices.reset_index().rename(
        columns={"date": "ds", "MA": "y"}
    )
    
    return prices

In [ ]:
def getIntradayPrices(crypto, n_days, training_amt, mov_avg, intra_int="5m"):
    """
    Gets crypto prices from now to N days ago and training amount will be in addition 
    to the number of days to train. (According to the EOD documentation: The maximum 
    periods between ‘from’ and ‘to’ are 120 days for 1-minute interval, 600 days 
    for 5-minute interval and 7200 days for 1 hour interval.)
    """